# Importanto Libs

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interact, Dropdown
import plotly.express as px
import plotly.graph_objects as go

# Lendo dataset

In [ ]:
df_merged = pd.read_csv('../data/processed/dataset_merged.csv')
df_merged

# Gráficos

## Número de pedidos por mês

In [ ]:
merged = df_merged[["customer_state", "year", "order_id",'year_month']].copy()

In [ ]:
col_estado = "customer_state"

# Agregar pedidos por estado, ano e ano_mes
pedidos = (
    merged.groupby([col_estado, "year", "year_month"], as_index=False)["order_id"]
    .nunique()
    .rename(columns={"order_id": "num_orders"})
)

In [ ]:
# listas para os dropdowns
estados = sorted(pedidos[col_estado].dropna().unique())
anos = sorted(pedidos["year"].dropna().unique())

dropdown_estado = widgets.Dropdown(
    options=estados,
    value=estados[0],
    description="Estado:",
    disabled=False
)

dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description="Ano:",
    disabled=False
)

output = widgets.Output()

def pedidos_mes(change=None):
    with output:
        output.clear_output()
        
        estado = dropdown_estado.value
        ano = dropdown_ano.value
        
        filtrado = pedidos[
            (pedidos[col_estado] == estado) &
            (pedidos["year"] == ano)
        ].sort_values("year_month")
        
        if filtrado.empty:
            print(f"Sem dados para {estado} em {ano}")
            return
        
        fig = px.line(
            filtrado,
            x="year_month",
            y="num_orders",
            markers=True,
            title=f"Pedidos por mês em {estado} no ano {ano}",
            labels={
                "year_month": "Ano-mês",
                "num_orders": "Número de pedidos"
            }
        )
        fig.update_layout(xaxis_tickangle=-45)
        fig.show()

# conectar os widgets à função
dropdown_estado.observe(pedidos_mes, names="value")
dropdown_ano.observe(pedidos_mes, names="value")

# mostrar os widgets e o gráfico
display(widgets.HBox([dropdown_estado, dropdown_ano]))
display(output)

pedidos_mes()


## Receita total por estado

In [ ]:
merged_2 = df_merged[["order_purchase_timestamp", "year", "customer_state",'payment_value']].copy()

In [ ]:
merged_2["order_purchase_timestamp"] = pd.to_datetime(merged_2["order_purchase_timestamp"])
merged_2["year"] = merged_2["order_purchase_timestamp"].dt.year

col_estado_2 = "customer_state"

receita = (
   merged_2
   .groupby([col_estado_2, "year"], as_index=False)["payment_value"]
   .sum()
   .rename(columns={"payment_value": "total_receita"})
)

In [ ]:
anos = sorted(receita["year"].unique())

dropdown_ano = widgets.Dropdown(
   options=anos,
   value=anos[0],
   description="Ano:",
   disabled=False
)


output = widgets.Output()

def receita_estado(change=None):
   with output:
      output.clear_output()
      
      ano = dropdown_ano.value
      
      filtrado = (
         receita[receita["year"] == ano]
         .sort_values("total_receita", ascending=False)
      )
      
      fig = px.bar(
         filtrado,
         x=col_estado,
         y="total_receita",
         title=f"Receita total por estado em {ano}",
         labels={
               col_estado: "Estado",
               "total_receita": "Receita Total (R$)",
         },
         color="customer_state",
         height=800,
         width=2000,
      )
      
      fig.update_layout(xaxis_tickangle=-45)
      fig.show()

dropdown_ano.observe(receita_estado, names="value")

display(dropdown_ano)
display(output)

# gerar gráfico inicial
receita_estado()

## Participação dos meios de pagamento

In [ ]:
merged_3 = df_merged[["payment_type",'payment_value','order_id']].copy()

In [ ]:
valor_limite = 500  # troca para o valor que quiser

qtd = len(merged_3[ merged_3["payment_value"] > valor_limite ])
print(qtd)


In [ ]:
meios_pagamento = sorted(merged_3["payment_type"].dropna().unique())


In [ ]:
dropdown_meio = widgets.Dropdown(
    options=meios_pagamento,
    value=meios_pagamento[0],
    description="Pagamento:"
)

In [ ]:
slider_intervalo = widgets.IntSlider(
    value=500,
    min=50,
    max=2000,
    step=50,
    description="Intervalo:"
)

In [ ]:
output = widgets.Output()

In [ ]:
# garantir que não tenha valores bizarros
merged_3 = merged_3[merged_3["payment_value"].notna() & (merged_3["payment_value"] > 0)]

meios = sorted(merged_3["payment_type"].dropna().unique())

dropdown_meio = widgets.Dropdown(
   options=meios,
   value=meios[0],
   description="Pagamento:"
)

slider_intervalo = widgets.IntSlider(
   value=500,
   min=50,
   max=2000,
   step=50,
   description="Intervalo:"
)

output = widgets.Output()

def participacao_meios_pagamentos(change=None):
   with output:
      clear_output()
      
      meio = dropdown_meio.value
      intervalo = slider_intervalo.value
      
      df_filtro = merged_3[merged_3["payment_type"] == meio].copy()
      
      if df_filtro.empty:
         print("Nenhum dado disponível para esse filtro.")
         return
      
      valor_max = df_filtro["payment_value"].max()
      bins = list(range(0, int(valor_max) + intervalo, intervalo))
      if len(bins) < 2:
         print("Poucos dados para formar faixas.")
         return
      
      labels = [f"{i} a {i+intervalo}" for i in bins[:-1]]
      
      df_filtro["faixa_valor"] = pd.cut(
         df_filtro["payment_value"],
         bins=bins,
         labels=labels,
         include_lowest=True
      )
      
      contagem = (
         df_filtro
         .groupby("faixa_valor")["order_id"]
         .nunique()
         .reset_index()
         .rename(columns={"order_id": "num_pedidos"})
      )
      
      contagem = contagem.sort_values("num_pedidos", ascending=False)
      
      fig = px.bar(
         contagem,
         x="faixa_valor",
         y="num_pedidos",
         title=f"Pedidos por faixa de valor (pagamento: {meio}, intervalo: R$ {intervalo})",
         labels={
               "faixa_valor": "Faixa de valor (R$)",
               "num_pedidos": "Número de pedidos",
               
         },
         color='faixa_valor',
         height=1000,
         width=1700,
         
      )
      
      fig.update_layout(xaxis_tickangle=-45)
      fig.show()

dropdown_meio.observe(participacao_meios_pagamentos, names="value")
slider_intervalo.observe(participacao_meios_pagamentos, names="value")

display(widgets.HBox([dropdown_meio, slider_intervalo]))
display(output)

participacao_meios_pagamentos()


## Relação entre valor do frete e valor do produto

In [ ]:
merged_4 = df_merged[["price", "freight_value", "customer_state"]].copy()

In [ ]:
merged_4

In [ ]:
import numpy as np
import pandas as pd

merged_4 = df_merged[["price", "freight_value", "customer_state"]].copy()

merged_4 = merged_4[
    merged_4["price"].notna() &
    merged_4["freight_value"].notna() &
    (merged_4["price"] > 0) &
    (merged_4["freight_value"] > 0)
].copy()

min_price = merged_4["price"].min()
max_price = merged_4["price"].max()

num_bins = 8  # você pode ajustar esse número

bins = np.linspace(min_price, max_price, num_bins + 1)
labels = [f"{int(bins[i])} a {int(bins[i+1])}" for i in range(len(bins) - 1)]

merged_4["faixa_preco"] = pd.cut(
    merged_4["price"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

estados = sorted(merged_4["customer_state"].dropna().unique())


In [ ]:
dropdown_estado = widgets.Dropdown(
    options=estados,
    value=estados[0],
    description="Estado:"
)

# cada opção é (label bonito, índice da faixa)
faixas = list(zip(labels, range(len(labels))))

slider_faixa = widgets.SelectionRangeSlider(
    options=faixas,
    index=(0, len(labels) - 1),   # começa pegando todas as faixas
    description="Faixas:",
    continuous_update=False
)

output = widgets.Output()


In [ ]:
def frete_produto(change=None):
   with output:
      clear_output()
      
      estado = dropdown_estado.value
      idx_min, idx_max = slider_faixa.value  # índices das faixas
      
      # limites reais em R$ a partir dos bins
      preco_min = bins[idx_min]
      preco_max = bins[idx_max + 1]  # +1 porque o fim da faixa é o bin seguinte
      
      df_estado = merged_4[
         (merged_4["customer_state"] == estado) &
         (merged_4["price"] >= preco_min) &
         (merged_4["price"] <= preco_max)
      ].copy()
      
      if df_estado.empty:
         print(
               f"Sem dados para {estado} entre as faixas "
               f"{labels[idx_min]} e {labels[idx_max]}"
         )
         return
      
      resumo = (
         df_estado
         .groupby("faixa_preco", observed=True)["freight_value"]
         .mean()
         .reset_index()
         .rename(columns={"freight_value": "frete_medio"})
      )
      
      resumo = resumo.dropna(subset=["faixa_preco"]).sort_values("faixa_preco")
      
      if resumo.empty:
         print("Não há faixas com dados dentro desse intervalo de faixas.")
         return
      
      fig = px.bar(
         resumo,
         x="faixa_preco",
         y="frete_medio",
         title=(
               f"Frete médio por faixa de preço em {estado} "
               f"(faixas de {labels[idx_min]} até {labels[idx_max]})"
         ),
         labels={
               "faixa_preco": "Faixa de preço do produto (R$)",
               "frete_medio": "Frete médio (R$)"
         },
         color='faixa_preco'
      )
      
      fig.update_layout(xaxis_tickangle=-45)
      fig.show()
      
dropdown_estado.observe(frete_produto, names="value")
slider_faixa.observe(frete_produto, names="value")

display(dropdown_estado)
display(slider_faixa)
display(output)

frete_produto()



## Média de avaliação dos pedidos por estado

In [ ]:
merged_5 = df_merged.copy()

In [ ]:
merged_5["order_purchase_timestamp"] = pd.to_datetime(merged_5["order_purchase_timestamp"])
merged_5["year"] = merged_5["order_purchase_timestamp"].dt.year

In [ ]:
df_reviews = merged_5[merged_5["review_score"].notna()].copy()

In [ ]:
base = (
   df_reviews
   .groupby(["customer_state", "year"], as_index=False)["review_score"]
   .mean()
   .rename(columns={"review_score": "avg_score"})
)


In [ ]:
anos = sorted(base["year"].unique())
estados = sorted(base["customer_state"].unique())

dropdown_ano = widgets.Dropdown(
   options=["Todos"] + anos,
   value="Todos",
   description="Ano:"
)

dropdown_status = widgets.Dropdown(
   options=["Todos", "Somente entregues"],
   value="Todos",
   description="Status:"
)

output = widgets.Output()


In [ ]:
def avaliacao_media_estado(change=None):
   with output:
      clear_output()
      
      ano_sel = dropdown_ano.value
      status_sel = dropdown_status.value

      df = df_merged.copy()

      # filtrar entregues se escolhido
      if status_sel == "Somente entregues":
         df = df[df["order_status"] == "delivered"]
      
      # filtrar por ano se selecionado
      if ano_sel != "Todos":
         df = df[df["year"] == ano_sel]
      
      # tirar avaliações ausentes
      df = df[df["review_score"].notna()].copy()
      
      if df.empty:
         print("Sem dados para esse filtro.")
         return
      
      resumo = (
         df.groupby("customer_state")["review_score"]
            .mean()
            .reset_index()
            .rename(columns={"review_score": "avg_score"})
      )
      
      resumo = resumo.sort_values("avg_score", ascending=False)
      
      fig = px.bar(
         resumo,
         x="customer_state",
         y="avg_score",
         title="Média de avaliação dos pedidos por estado",
         labels={
               "customer_state": "Estado",
               "avg_score": "Avaliação média"
         },
         color="avg_score",
         color_continuous_scale="Bluered",
         width=2000
         
      )
      
      fig.update_layout(xaxis_tickangle=-45)
      fig.show()
      
dropdown_ano.observe(avaliacao_media_estado, names="value")
dropdown_status.observe(avaliacao_media_estado, names="value")

display(widgets.HBox([dropdown_ano, dropdown_status]))
display(output)

avaliacao_media_estado()

## Mapa

In [ ]:
merged_6 = df_merged.copy()

In [ ]:
# garantir year
merged_6["order_purchase_timestamp"] = pd.to_datetime(merged_6["order_purchase_timestamp"])
merged_6["year"] = merged_6["order_purchase_timestamp"].dt.year

# agregação por estado e ano
map_base = (
   merged_6
   .groupby(["customer_state", "year"], as_index=False)
   .agg(
      num_orders=("order_id", "nunique"),
      total_revenue=("payment_value", "sum")
   )
)

# lista fixa de UFs
ufs = [
   "AC", "AL", "AP", "AM", "BA", "CE",
   "DF", "ES", "GO", "MA", "MT", "MS",
   "MG", "PA", "PB", "PR", "PE", "PI",
   "RJ", "RN", "RS", "RO", "RR", "SC",
   "SP", "SE", "TO"
]
df_estados_full = pd.DataFrame({"customer_state": ufs})



In [ ]:
br_states = '../data/brazil-states.geojson'
code_field = "sigla"

In [ ]:

anos = sorted(map_base["year"].unique())

dropdown_ano = widgets.Dropdown(
   options=anos,
   value=anos[0],
   description="Ano:"
)

output = widgets.Output()

def mapa(change=None):
   with output:
      clear_output()
      
      ano_sel = dropdown_ano.value
      
      # dados desse ano
      df_ano_raw = map_base[map_base["year"] == ano_sel].copy()
      
      # garantir que todos os estados existam, mesmo sem dados
      df_ano = df_estados_full.merge(df_ano_raw, on="customer_state", how="left")
      
      # coluna usada no mapa: NaN vira 0 (sem pedidos)
      df_ano["num_orders_plot"] = df_ano["num_orders"].fillna(0)
      
      # para definir o range da escala de cores
      max_orders = df_ano["num_orders_plot"].max()
      
      # colorscale contínuo em que 0 é cinza e o resto é Viridis
      viridis = px.colors.sequential.Viridis
      colorscale = [
         [0.0, "#D3D3D3"],       # 0 pedidos -> cinza claro
         [1e-99999, viridis[0]],    # logo acima de zero já entra na paleta
         [1.0, viridis[-1]],    # máximo -> cor mais intensa
      ]
      
      fig = px.choropleth(
         df_ano,
         geojson=br_states,
         locations="customer_state",
         featureidkey=f"properties.{code_field}",
         color="num_orders_plot",
         color_continuous_scale=colorscale,
         range_color=(0, max_orders),
         hover_name="customer_state",
         hover_data={
               "num_orders": True,
               "total_revenue": ":,.2f"
         }
      )

      fig.update_geos(
         fitbounds="locations",
         visible=False
      )

      fig.update_layout(
         title=f"Número de pedidos e receita por estado em {ano_sel}",
         margin={"r": 0, "t": 40, "l": 0, "b": 0},
         height=1000
      )

      fig.show()

dropdown_ano.observe(mapa, names="value")

display(dropdown_ano)
display(output)

mapa()